<a href="https://colab.research.google.com/github/Alex-Jung-HB/0724_Python_Video-analysis-with-trained-codes-of-Roboflow/blob/main/0724_Python_Video_analysis_with_trained_codes_of_Roboflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Video analysis by Traffic sign trained codes from Roboflow of 명민호

In [7]:
# =============================================================================
# FIXED LANE DETECTION WITH ROBOFLOW - PROPER ERROR HANDLING
# =============================================================================

import cv2
import os
from roboflow import Roboflow
import numpy as np
import yt_dlp
import subprocess
import sys

# Auto-install packages if needed
def install_packages():
    """🔧 자동으로 필요한 패키지 설치"""
    packages = ['opencv-python', 'roboflow', 'ultralytics', 'yt-dlp', 'numpy']

    for package in packages:
        try:
            __import__(package.replace('-', '_'))
        except ImportError:
            print(f"📦 Installing {package}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Install packages
install_packages()

# File selection utilities
try:
    import tkinter as tk
    from tkinter import filedialog
    GUI_AVAILABLE = True
except ImportError:
    GUI_AVAILABLE = False

try:
    from google.colab import files
    COLAB_AVAILABLE = True
except ImportError:
    COLAB_AVAILABLE = False

# =============================================================================
# CONFIGURATION - YOUR TRAINED MODEL
# =============================================================================

# 🔑 Your API Key
API_KEY = "4Y7PE5GMkG6cfSh3pUqT"  # ✅ YOUR CONFIRMED API KEY

# 🎯 YOUR TRAINED MODEL CONFIGURATION (FROM UNIVERSE)
class ModelConfig:
    """✅ 확인된 훈련된 모델 정보"""

    # ✅ Your working Universe model details
    UNIVERSE_WORKSPACE = "pythoneducation"
    UNIVERSE_PROJECT = "road-wu7gh-q6zrb"
    UNIVERSE_VERSION = "1"
    UNIVERSE_FULL_PATH = "pythoneducation/road-wu7gh-q6zrb"

    # 📊 Model Performance (from your Universe page)
    MODEL_TYPE = "YOLOv11 Object Detection (Fast)"
    CHECKPOINT = "COCOn"
    MAP_50 = "66.0%"
    PRECISION = "68.3%"
    RECALL = "55.0%"
    TRAINING_IMAGES = 432

    # 🎯 Known detectable objects (from your model)
    DETECTABLE_OBJECTS = ["green light", "traffic light", "road signs"]

    # Option 2: Alternative pre-trained models (if yours doesn't work)
    ALTERNATIVE_MODELS = [
        {
            "name": "Vehicle Detection Model",
            "workspace": "roboflow-100",
            "project": "vehicles-q0x2v",
            "version": "2",
            "description": "차량 감지 모델 (도로 관련)"
        },
        {
            "name": "Traffic Signs Detection",
            "workspace": "jacob-solawetz",
            "project": "traffic-eqlkl",
            "version": "1",
            "description": "교통 표지판 감지 모델"
        },
        {
            "name": "Road Objects Detection",
            "workspace": "brad-dwyer",
            "project": "road-objects-8bm38",
            "version": "1",
            "description": "도로 객체 감지 모델"
        }
    ]

def load_lane_detection_model():
    """
    🛠️ 확인된 훈련된 모델 로드
    """
    print("🔑 Roboflow에 연결 중...")
    rf = Roboflow(api_key=API_KEY)

    # Method 1: Load your confirmed working model
    print(f"\n✅ 확인된 모델 로드 중...")
    print(f"   📂 프로젝트: {ModelConfig.UNIVERSE_WORKSPACE}/{ModelConfig.UNIVERSE_PROJECT}")
    print(f"   🔢 버전: {ModelConfig.UNIVERSE_VERSION}")
    print(f"   🎯 모델 타입: {ModelConfig.MODEL_TYPE}")
    print(f"   📊 성능: mAP@50={ModelConfig.MAP_50}, Precision={ModelConfig.PRECISION}")

    try:
        # Try Universe access (your model is public)
        print("🌐 Universe에서 모델 로드 중...")
        model = rf.universe().project(ModelConfig.UNIVERSE_FULL_PATH).version(ModelConfig.UNIVERSE_VERSION).model

        print("✅ 모델 로드 성공!")
        print(f"🎯 감지 가능한 객체: {', '.join(ModelConfig.DETECTABLE_OBJECTS)}")
        print(f"📚 훈련 이미지 수: {ModelConfig.TRAINING_IMAGES}개")
        return model, "your_trained_model"

    except Exception as e:
        print(f"❌ Universe 접근 실패, 워크스페이스 접근 시도: {e}")

        # Fallback: Try workspace access
        try:
            print("🏢 워크스페이스에서 모델 로드 중...")
            project = rf.workspace(ModelConfig.UNIVERSE_WORKSPACE).project(ModelConfig.UNIVERSE_PROJECT)
            model = project.version(int(ModelConfig.UNIVERSE_VERSION)).model

            print("✅ 워크스페이스에서 모델 로드 성공!")
            return model, "your_trained_model"

        except Exception as e2:
            print(f"❌ 워크스페이스 접근도 실패: {e2}")

    # Method 2: Try alternative models (as backup)
    print(f"\n🔄 백업 모델들 시도 중...")

    alternative_models = [
        {
            "name": "Traffic Detection Model",
            "workspace": "roboflow-100",
            "project": "vehicles-q0x2v",
            "version": "2",
            "description": "차량 및 교통 관련 객체 감지"
        }
    ]

    for alt_model in alternative_models:
        print(f"\n   🧪 백업 모델 시도: {alt_model['name']}")
        try:
            model = rf.universe().project(f"{alt_model['workspace']}/{alt_model['project']}").version(alt_model['version']).model
            print(f"✅ 백업 모델 로드 성공: {alt_model['name']}")
            return model, "backup_model"
        except Exception as e:
            print(f"❌ 백업 모델 실패: {str(e)[:50]}...")

    return None, None

def load_specific_model(rf, workspace, project, version):
    """
    🎯 특정 모델 로드 시도
    """
    try:
        # Try Universe access first
        try:
            model = rf.universe().project(f"{workspace}/{project}").version(version).model
            return model
        except:
            # Try workspace access
            project_obj = rf.workspace(workspace).project(project)

            # Check if model has trained versions
            versions = project_obj.versions()
            if len(versions) == 0:
                raise Exception("모델이 아직 훈련되지 않았습니다")

            # Use the latest version or specified version
            if version.isdigit():
                model = project_obj.version(int(version)).model
            else:
                model = versions[0].model  # Use latest

            return model
    except Exception as e:
        raise Exception(f"모델 로드 실패: {e}")

def check_model_training_status():
    """
    🔍 모델 훈련 상태 확인
    """
    print("🔍 모델 훈련 상태 확인 중...")
    print("=" * 50)

    try:
        rf = Roboflow(api_key=API_KEY)

        # Check your specific project
        workspace = ModelConfig.UNIVERSE_WORKSPACE
        project = ModelConfig.UNIVERSE_PROJECT

        print(f"📂 프로젝트 확인: {workspace}/{project}")

        try:
            project_obj = rf.workspace(workspace).project(project)
            versions = project_obj.versions()

            print(f"📊 발견된 버전 수: {len(versions)}")

            if len(versions) == 0:
                print("❌ 훈련된 모델이 없습니다!")
                print("\n🚀 해결 방법:")
                print(f"1. Roboflow 프로젝트 페이지로 이동:")
                print(f"   https://app.roboflow.com/{workspace}/{project}")
                print("2. 'Train' 버튼 클릭")
                print("3. 훈련 설정 확인 후 훈련 시작")
                print("4. 훈련 완료까지 대기 (보통 5-30분)")
                print("5. 훈련 완료 후 이 스크립트 다시 실행")
                return False
            else:
                print("✅ 훈련된 모델이 있습니다!")
                for i, version in enumerate(versions):
                    status = getattr(version, 'status', 'Unknown')
                    print(f"   버전 {version.version}: 상태 - {status}")
                return True

        except Exception as pe:
            print(f"❌ 프로젝트 접근 실패: {pe}")
            print("\n💡 가능한 원인:")
            print("1. API 키가 잘못되었거나 권한이 없음")
            print("2. 프로젝트 이름이 잘못되었음")
            print("3. 프로젝트가 비공개로 설정되었음")
            return False

    except Exception as e:
        print(f"❌ API 연결 실패: {e}")
        return False

def download_video_with_fallback(urls):
    """
    📥 여러 URL로 비디오 다운로드 시도
    """
    for i, url in enumerate(urls):
        print(f"🎬 비디오 다운로드 시도 {i+1}/{len(urls)}: {url}")

        ydl_opts = {
            'format': 'best[height<=720]',
            'outtmpl': './driving_video.%(ext)s',
            'user_agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
            'sleep_interval': 1,
            'max_sleep_interval': 3,
        }

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            try:
                ydl.download([url])
                print("✅ 비디오 다운로드 성공!")
                return True
            except Exception as e:
                print(f"❌ 다운로드 실패: {str(e)[:50]}...")

    return False

def select_local_video():
    """
    📁 로컬 비디오 파일 선택
    """
    if COLAB_AVAILABLE:
        print("📁 Google Colab 파일 업로드...")
        uploaded = files.upload()
        if uploaded:
            filename = list(uploaded.keys())[0]
            return filename
    elif GUI_AVAILABLE:
        print("📁 파일 선택 창 열기...")
        root = tk.Tk()
        root.withdraw()
        file_path = filedialog.askopenfilename(
            title="비디오 파일 선택",
            filetypes=[('비디오 파일', '*.mp4 *.avi *.mov *.mkv *.wmv')]
        )
        root.destroy()
        return file_path
    else:
        print("📝 파일 경로를 직접 입력하세요:")
        return input("파일 경로: ").strip().strip('"\'')

def detect_objects_in_video(video_path, model, model_type, output_path="traffic_detection_result.mp4"):
    """
    🔍 비디오에서 교통 관련 객체 감지 (교통신호등, 표지판 등)
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("❌ 비디오 파일을 열 수 없습니다.")
        return False

    # 비디오 정보
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"🎬 비디오 정보: {width}x{height}, {fps}fps, {total_frames}프레임")

    # 출력 설정
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0
    total_detections = 0

    # 교통 객체별 색상 설정
    traffic_colors = {
        'green light': (0, 255, 0),       # 초록색 - 초록 신호등
        'red light': (0, 0, 255),         # 빨간색 - 빨간 신호등
        'yellow light': (0, 255, 255),    # 노란색 - 노란 신호등
        'traffic light': (255, 0, 255),   # 보라색 - 일반 신호등
        'stop sign': (0, 0, 255),         # 빨간색 - 정지 표지판
        'speed limit': (255, 165, 0),     # 주황색 - 속도 제한 표지판
        'your_trained_model': (0, 255, 0), # 초록색 - 훈련된 모델
        'backup_model': (255, 255, 0)     # 노란색 - 백업 모델
    }

    print("🚦 교통 객체 감지 시작...")
    print(f"🎯 감지 대상: {', '.join(ModelConfig.DETECTABLE_OBJECTS)}")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # 매 2프레임마다 감지 수행 (성능 최적화)
        if frame_count % 2 == 0:
            try:
                temp_img_path = "temp_frame.jpg"
                cv2.imwrite(temp_img_path, frame)

                # 신뢰도 설정 (훈련된 모델은 더 낮은 임계값 사용)
                confidence = 40 if model_type == "your_trained_model" else 50

                # 교통 객체 감지
                prediction = model.predict(temp_img_path, confidence=confidence, overlap=50)
                predictions = prediction.json()['predictions']

                frame_detections = len(predictions)
                total_detections += frame_detections

                if frame_detections > 0:
                    detected_objects = [pred.get('class', 'Unknown') for pred in predictions]
                    print(f"🚦 프레임 {frame_count}: {frame_detections}개 객체 감지 - {', '.join(set(detected_objects))}")

                # 감지된 교통 객체 그리기
                for detection in predictions:
                    x1 = int(detection['x'] - detection['width']/2)
                    y1 = int(detection['y'] - detection['height']/2)
                    x2 = int(detection['x'] + detection['width']/2)
                    y2 = int(detection['y'] + detection['height']/2)

                    # 객체 타입에 따른 색상 선택
                    class_name = detection.get('class', 'unknown').lower()
                    color = traffic_colors.get(class_name, traffic_colors.get(model_type, (255, 0, 255)))

                    # 바운딩 박스 그리기 (두껍게)
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)

                    # 라벨 표시 (개선된 스타일)
                    confidence_score = detection['confidence']
                    label = f"{detection.get('class', 'Object')}: {confidence_score:.1f}%"

                    # 라벨 배경 (더 큰 박스)
                    label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)[0]
                    cv2.rectangle(frame, (x1, y1-35), (x1+label_size[0]+15, y1), color, -1)

                    # 라벨 텍스트 (더 큰 폰트)
                    cv2.putText(frame, label, (x1+7, y1-10),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

                    # 신뢰도가 높은 경우 추가 강조
                    if confidence_score > 80:
                        cv2.circle(frame, (x2-10, y1+10), 5, (0, 255, 0), -1)  # 초록색 원

                if os.path.exists(temp_img_path):
                    os.remove(temp_img_path)

            except Exception as e:
                print(f"❌ 프레임 {frame_count} 처리 오류: {e}")

        out.write(frame)

        # 진행률 표시 (더 자주)
        if frame_count % 100 == 0:
            progress = (frame_count / total_frames) * 100
            print(f"📊 진행률: {progress:.1f}% (총 감지: {total_detections}개)")

    cap.release()
    out.release()

    print(f"✅ 교통 객체 감지 완료!")
    print(f"📊 총 {total_detections}개 교통 객체 감지")
    print(f"🎥 결과 파일: {output_path}")
    print(f"💡 주요 감지 객체: 교통신호등, 도로 표지판")
    return True

def main_detection_pipeline():
    """
    🚀 교통 객체 감지 파이프라인 (확인된 훈련된 모델 사용)
    """
    print("🚦 교통 신호등 및 표지판 감지 프로그램 시작!")
    print("=" * 60)
    print("✅ 확인된 훈련된 모델 사용:")
    print(f"   📂 프로젝트: {ModelConfig.UNIVERSE_WORKSPACE}/{ModelConfig.UNIVERSE_PROJECT}")
    print(f"   🎯 감지 대상: {', '.join(ModelConfig.DETECTABLE_OBJECTS)}")
    print(f"   📊 모델 성능: mAP@50={ModelConfig.MAP_50}")
    print("=" * 60)

    # Step 1: Load your trained model
    print("🤖 STEP 1: 훈련된 모델 로드...")
    model, model_type = load_lane_detection_model()

    if not model:
        print("❌ 모델 로드에 실패했습니다.")
        print("\n🔧 해결 방법:")
        print("1. 인터넷 연결 확인")
        print("2. API 키 확인")
        print("3. Roboflow Universe 페이지 확인:")
        print(f"   https://universe.roboflow.com/{ModelConfig.UNIVERSE_FULL_PATH}/model/{ModelConfig.UNIVERSE_VERSION}")
        return

    print(f"✅ 모델 로드 성공! (타입: {model_type})")

    # Step 2: Get video
    print("\n📹 STEP 2: 비디오 준비...")
    print("🚦 교통 신호등이나 표지판이 있는 비디오를 선택하세요")
    print("1️⃣ YouTube에서 교통 영상 다운로드")
    print("2️⃣ 로컬 파일 선택")

    choice = input("선택하세요 (1 또는 2): ").strip()
    video_path = None

    if choice == "1":
        # Traffic-focused YouTube URLs
        traffic_youtube_urls = [
            "https://www.youtube.com/watch?v=2mWPGoOzKBM",  # 도시 교통 영상
            "https://www.youtube.com/watch?v=7QYJALqOCjU",  # 교차로 영상
            "https://www.youtube.com/watch?v=dv13gl0a-FA",  # 대시캠 영상
            "https://www.youtube.com/watch?v=1uJgXwc3Ja8",  # 시내 주행
        ]

        print("🚦 교통 관련 영상 다운로드 중...")
        if download_video_with_fallback(traffic_youtube_urls):
            video_files = [f for f in os.listdir('.') if f.startswith('driving_video')]
            if video_files:
                video_path = video_files[0]

        if not video_path:
            print("❌ YouTube 다운로드 실패. 로컬 파일을 선택해주세요.")
            video_path = select_local_video()

    elif choice == "2":
        video_path = select_local_video()

    if not video_path or not os.path.exists(video_path):
        print("❌ 유효한 비디오 파일을 찾을 수 없습니다.")
        return

    # Step 3: Run traffic detection
    print(f"\n🚦 STEP 3: 교통 객체 감지 시작...")
    print(f"📁 처리할 파일: {os.path.basename(video_path)}")
    print(f"🎯 감지 대상: 교통신호등, 도로 표지판")

    output_filename = f"traffic_detection_result_{model_type}.mp4"
    success = detect_objects_in_video(video_path, model, model_type, output_filename)

    if success:
        print("\n🎉 교통 객체 감지 완료!")
        print(f"📺 결과 파일: {output_filename}")
        print("\n📊 결과 확인 가이드:")
        print("✅ 초록색 박스: 초록 신호등")
        print("🔴 빨간색 박스: 빨간 신호등/정지 표지판")
        print("🟡 노란색 박스: 노란 신호등")
        print("🟣 보라색 박스: 일반 교통 신호등")
        print("🟠 주황색 박스: 속도 제한 표지판")
        print("\n💡 팁:")
        print("- 감지 성능이 낮으면 더 선명한 교통 영상을 사용해보세요")
        print("- 낮이나 밝은 환경에서 촬영된 영상이 더 좋은 결과를 보입니다")
        print(f"- 현재 모델 성능: 정밀도 {ModelConfig.PRECISION}, 재현율 {ModelConfig.RECALL}")
    else:
        print("❌ 교통 객체 감지 중 오류가 발생했습니다.")

# =============================================================================
# EXECUTION
# =============================================================================
if __name__ == "__main__":
    print("🚦 교통 신호등 및 표지판 감지 프로그램")
    print("=" * 60)
    print("✅ 확인된 훈련된 Roboflow 모델 사용")
    print(f"🌐 모델 URL: https://universe.roboflow.com/{ModelConfig.UNIVERSE_FULL_PATH}/model/{ModelConfig.UNIVERSE_VERSION}")
    print("📊 모델 정보:")
    print(f"   • 모델 타입: {ModelConfig.MODEL_TYPE}")
    print(f"   • 훈련 이미지: {ModelConfig.TRAINING_IMAGES}개")
    print(f"   • 성능: mAP@50={ModelConfig.MAP_50}, 정밀도={ModelConfig.PRECISION}")
    print(f"   • 감지 객체: {', '.join(ModelConfig.DETECTABLE_OBJECTS)}")
    print("=" * 60)
    print("🎯 이 프로그램의 기능:")
    print("   ✅ 교통 신호등 감지 (빨강, 노랑, 초록)")
    print("   ✅ 도로 표지판 감지")
    print("   ✅ YouTube 비디오 또는 로컬 파일 처리")
    print("   ✅ 감지 결과를 새 비디오로 저장")
    print("=" * 60)
    print("💡 최적의 결과를 위한 팁:")
    print("   • 낮시간 또는 밝은 환경의 영상 사용")
    print("   • 교통 신호등이 명확히 보이는 영상 선택")
    print("   • 해상도가 너무 낮지 않은 영상 사용")
    print("=" * 60)

    try:
        main_detection_pipeline()
    except KeyboardInterrupt:
        print("\n⏹️ 사용자에 의해 프로그램이 중단되었습니다.")
    except Exception as e:
        print(f"\n❌ 예상치 못한 오류 발생: {e}")
        print("🔧 문제 해결:")
        print("1. 인터넷 연결 확인")
        print("2. API 키 확인")
        print(f"3. Roboflow 모델 페이지 확인: https://universe.roboflow.com/{ModelConfig.UNIVERSE_FULL_PATH}")
        print("4. 비디오 파일 형식 확인 (mp4, avi, mov 등)")

📦 Installing opencv-python...
🚦 교통 신호등 및 표지판 감지 프로그램
✅ 확인된 훈련된 Roboflow 모델 사용
🌐 모델 URL: https://universe.roboflow.com/pythoneducation/road-wu7gh-q6zrb/model/1
📊 모델 정보:
   • 모델 타입: YOLOv11 Object Detection (Fast)
   • 훈련 이미지: 432개
   • 성능: mAP@50=66.0%, 정밀도=68.3%
   • 감지 객체: green light, traffic light, road signs
🎯 이 프로그램의 기능:
   ✅ 교통 신호등 감지 (빨강, 노랑, 초록)
   ✅ 도로 표지판 감지
   ✅ YouTube 비디오 또는 로컬 파일 처리
   ✅ 감지 결과를 새 비디오로 저장
💡 최적의 결과를 위한 팁:
   • 낮시간 또는 밝은 환경의 영상 사용
   • 교통 신호등이 명확히 보이는 영상 선택
   • 해상도가 너무 낮지 않은 영상 사용
🚦 교통 신호등 및 표지판 감지 프로그램 시작!
✅ 확인된 훈련된 모델 사용:
   📂 프로젝트: pythoneducation/road-wu7gh-q6zrb
   🎯 감지 대상: green light, traffic light, road signs
   📊 모델 성능: mAP@50=66.0%
🤖 STEP 1: 훈련된 모델 로드...
🔑 Roboflow에 연결 중...

✅ 확인된 모델 로드 중...
   📂 프로젝트: pythoneducation/road-wu7gh-q6zrb
   🔢 버전: 1
   🎯 모델 타입: YOLOv11 Object Detection (Fast)
   📊 성능: mAP@50=66.0%, Precision=68.3%
🌐 Universe에서 모델 로드 중...
❌ Universe 접근 실패, 워크스페이스 접근 시도: 'Roboflow' object has no attribute 'universe'
🏢 워크스페이스에서 모델 로드 중..

Saving KakaoTalk_20250717_091801377.mp4 to KakaoTalk_20250717_091801377.mp4

🚦 STEP 3: 교통 객체 감지 시작...
📁 처리할 파일: KakaoTalk_20250717_091801377.mp4
🎯 감지 대상: 교통신호등, 도로 표지판
🎬 비디오 정보: 1600x720, 24fps, 1703프레임
🚦 교통 객체 감지 시작...
🎯 감지 대상: green light, traffic light, road signs
🚦 프레임 2: 1개 객체 감지 - zebrapad
🚦 프레임 20: 2개 객체 감지 - zebrapad
🚦 프레임 38: 1개 객체 감지 - zebrapad
🚦 프레임 40: 1개 객체 감지 - zebrapad
🚦 프레임 56: 1개 객체 감지 - zebrapad
🚦 프레임 62: 1개 객체 감지 - zebrapad
🚦 프레임 80: 1개 객체 감지 - Stop-bord
🚦 프레임 82: 2개 객체 감지 - zebrapad, Stop-bord
🚦 프레임 84: 1개 객체 감지 - Stop-bord
🚦 프레임 96: 1개 객체 감지 - Stop-bord
📊 진행률: 5.9% (총 감지: 12개)
🚦 프레임 134: 2개 객체 감지 - zebrapad
🚦 프레임 190: 1개 객체 감지 - zebrapad
📊 진행률: 11.7% (총 감지: 15개)
🚦 프레임 282: 1개 객체 감지 - zebrapad
🚦 프레임 298: 1개 객체 감지 - zebrapad
📊 진행률: 17.6% (총 감지: 17개)
📊 진행률: 23.5% (총 감지: 17개)
🚦 프레임 434: 1개 객체 감지 - voorrangweg
📊 진행률: 29.4% (총 감지: 18개)
📊 진행률: 35.2% (총 감지: 18개)
🚦 프레임 680: 1개 객체 감지 - zebrapad
📊 진행률: 41.1% (총 감지: 19개)
📊 진행률: 47.0% (총 감지: 19개)
🚦 프레임 842: 1개 객체 감지 - zebrapad
